In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
import datetime as dt
import os

import warnings
warnings.filterwarnings('ignore')

In [61]:
def fit_arima_garch(data, arima_order=(1, 1, 1), garch_order=(1, 1)):
    # Fit ARIMA model
    arima_model = sm.tsa.arima.ARIMA(data, order=arima_order)
    arima_res = arima_model.fit()

    # Obtain ARIMA model residuals
    arima_residuals = arima_res.resid

    # Fit GARCH model on ARIMA residuals
    garch_model = arch_model(arima_residuals, vol='GARCH', p=garch_order[0], q=garch_order[1])
    garch_res = garch_model.fit(disp='off')

    return arima_res, garch_res

def predict_arima_garch(arima_res, garch_res, steps=1):
    # Forecast ARIMA
    arima_forecast = arima_res.forecast(steps=steps)

    # Forecast GARCH
    garch_forecast = garch_res.forecast(start=len(arima_res.resid) - 1, horizon=steps)

    # Get GARCH conditional variances
    garch_cond_var = garch_forecast.variance.values[-1, :]

    # Combine ARIMA forecasts and GARCH conditional variances
    combined_forecast = arima_forecast + np.sqrt(garch_cond_var)

    return combined_forecast



In [58]:
# import data
######### import data #########
main_path = os.getcwd()

#load price (exchange rate) data
df_P = pd.read_pickle(main_path+'/data/future_price_use.pkl')
df_P = df_P.fillna(method='ffill').drop(df_P.index[0])

#weekly return
df_r = pd.read_pickle(main_path+'/data/future_return_use.pkl')
df_r = df_r.fillna(method='ffill').dropna()

df_r2 = pd.read_pickle(main_path + "/data/future_return_bear.pkl")
df_r2 = df_r2.fillna(method='ffill').dropna()

In [107]:
# Array of asset names
assets = list(df_r2.columns)

# A new dataframe to store my predictions
my_preds = pd.DataFrame(columns=df_r2.columns)

window_size = 105

# We make n_preds predictions
n_preds = df_r2.shape[0] - window_size + 1

arima_para = {'GC': (3,1,1), 'HE': (4,1,2), 'ZC': (3,1,1), 'CT': (4,1,1), 'YM': (5,1,1), 'NG':(4,1,1)}



# n_preds = 50

for i in range(n_preds):
    # Train data
    train_data = df_r2.iloc[i:i + window_size]
    # print(train_data[0:].index[0], i)

    # Predict asset return for one step
    for a in assets:
        # arima and garch models
        arima_res, garch_res = fit_arima_garch(train_data[a], arima_order=(1,1,1), garch_order=(1, 1))
        # use these models to predict asset prices
        combined_forecast = predict_arima_garch(arima_res, garch_res, steps = 1)
        # t = combined_forecast.index[0]
        my_preds.loc[window_size+i,a] = combined_forecast.values[0]
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96


In [108]:
my_preds

,GC,ZC,HE,CT,YM,NG
105,0.021983,0.040392,0.037865,0.030964,0.016968,0.046695
106,0.021398,0.031685,0.034015,0.032676,0.015681,0.051143
107,0.022019,0.035649,0.029843,0.036007,0.01725,0.041386
108,0.022839,0.060979,0.048046,0.031747,0.015637,0.054095
109,0.022128,0.035893,0.053338,0.033378,0.018968,0.044473
...,...,...,...,...,...,...
197,0.019097,0.039775,0.055229,0.025574,0.015006,0.032136
198,0.0173,0.037913,0.054335,0.027914,0.013387,0.033585
199,0.016695,0.038592,0.049514,0.030204,0.014219,0.029851
200,0.018869,0.03792,0.062416,0.026327,0.015661,0.040794


In [109]:
# Save predictions
folder_path = main_path+'/data'
file_path = os.path.join(folder_path, 'ccm_preds_bear.pkl')
my_preds.to_pickle(file_path)

In [38]:
# Using auto_arima to find parameters for each asset
from pmdarima.arima import auto_arima, ADFTest

arima_model = auto_arima(df_r['NG'], start_p = 0,d = 1, start_q = 0, max_p = 5, max_d = 5, max_q = 5, start_P = 0, D = 1, start_Q = 0, max_P = 5, max_D = 5, max_Q = 5, m = 12, seasonal = False, error_action = 'warn', trace = True, supress_warnings = True, stepwise = True, random_state = 20, n_fits = 50)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-406.384, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-471.561, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-408.384, Time=0.03 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-511.058, Time=0.10 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-513.957, Time=0.07 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=-521.140, Time=0.09 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=-524.963, Time=0.11 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : AIC=-558.003, Time=0.31 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=-560.466, Time=0.30 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.21 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=-557.166, Time=0.30 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-560.081, Time=0.25 sec
 ARIMA(5,1,2)(0,0,0)[0] intercept   : AIC=-553.697, Time=0.33 sec
 ARIMA(4,1,1)(0,0,0)[0]             : AIC=i

In [110]:
my_preds

,GC,ZC,HE,CT,YM,NG
105,0.021983,0.040392,0.037865,0.030964,0.016968,0.046695
106,0.021398,0.031685,0.034015,0.032676,0.015681,0.051143
107,0.022019,0.035649,0.029843,0.036007,0.01725,0.041386
108,0.022839,0.060979,0.048046,0.031747,0.015637,0.054095
109,0.022128,0.035893,0.053338,0.033378,0.018968,0.044473
...,...,...,...,...,...,...
197,0.019097,0.039775,0.055229,0.025574,0.015006,0.032136
198,0.0173,0.037913,0.054335,0.027914,0.013387,0.033585
199,0.016695,0.038592,0.049514,0.030204,0.014219,0.029851
200,0.018869,0.03792,0.062416,0.026327,0.015661,0.040794
